In [54]:
# import modules
import requests
import json
import time
import os
from dotenv import load_dotenv # Need to use this in anaconda prompt: pip3 install python-dotenv

# For displaying the images
import pprint
import json

# load .env file
load_dotenv("APIKey.env")

# get api key from .env file
key = os.environ.get("key")

# Getting the Model id from the .env file. The specific Model id. Insert it here.
model_id = os.environ.get("model_id")

# If the code is to be used elsewhere, insert the API key here instead of the variable.
API_KEY = key

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {API_KEY}"
}

In [11]:
def get_model_version(model_id, version_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/versions/{version_id}"
    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    version_id = data["id"]
    status = data["status"]

    print(f"Version ID: {version_id}. Status: {status}")

    return version_id, status

In [12]:
def generate_image(model_id, prompt):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

    payload = {
        "prompt": prompt,
        "steps": 50,
        "width": 512,
        "height": 512,
        "numberOfImages": 2,
        "seed": 4523184
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    status = data["status"]

    print(f"Inference ID: {inference_id}. Status: {status}")

    return inference_id, status

In [13]:
def get_inference_job(model_id, inference_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences/{inference_id}"

    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    state = data["state"]
    image = None

    if len(data["images"]):
        image = data["images"][0]["uri"]

    print(f"Inference ID: {inference_id}. State: {state}")

    return inference_id, state, image

In [51]:
# Now that we have a fine-tuned version of a model, we can
# generate images using it. Notice how I'm using '@me' to 
# indicate I want pictures similar to the ones we upload to 

# Prompt text. Insert text here. Remember to use the @
promptText = "A photo of @Andreas with a cigar, sunglasses, hawaii shirt, and a sombrero"

inference_id, status = generate_image(
    model_id, 
    prompt = promptText
)
while status != "finished":
    time.sleep(10)
    inference_id, status, image = get_inference_job(model_id, inference_id)

# Only one of the examples are printed here. For all versions go check the code block below
print(image)

Inference ID: c2ce7699-963e-466b-b269-585936b7b335. Status: queued
Inference ID: c2ce7699-963e-466b-b269-585936b7b335. State: queued
Inference ID: c2ce7699-963e-466b-b269-585936b7b335. State: queued
Inference ID: c2ce7699-963e-466b-b269-585936b7b335. State: queued
Inference ID: c2ce7699-963e-466b-b269-585936b7b335. State: finished
ng


In [52]:
# Listing images, see API: https://www.tryleap.ai/projects/ba2ce5ea-8721-4153-ad05-a6eafbdcbb4f/models/docs

urlList = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

response = requests.get(urlList, headers=HEADERS)

# ordering the list with new objects on new lines. Printing the latest 3 versions
ImageList = json.loads(response.text)
pprint.pprint(ImageList[-3:])


[{'createdAt': '2023-03-20T08:37:40.37697+00:00',
  'height': 512,
  'id': '7468249e-edd3-4d9a-a649-79c8159ce86d',
  'images': [{'createdAt': '2023-03-20T08:38:04.727Z',
              'id': 'dd771392-92f3-4f28-ae2f-800b1cf8128c',
              'uri': 'https://dreamtrain.s3.us-west-2.amazonaws.com/image-gen-7468249e-edd3-4d9a-a649-79c8159ce86d/generated_images/1.png'},
             {'createdAt': '2023-03-20T08:38:04.727Z',
              'id': 'ecfb6c22-4dca-4b15-be96-eb2973d591e6',
              'uri': 'https://dreamtrain.s3.us-west-2.amazonaws.com/image-gen-7468249e-edd3-4d9a-a649-79c8159ce86d/generated_images/0.png'}],
  'modelId': '1cc96b76-92f7-4253-9092-bfb5d5de7d52',
  'negativePrompt': '',
  'numberOfImages': 2,
  'prompt': 'a photo of ynmfenw91g person with sunglasses, a cigar, hawaii '
            'shirt, and a sombrero',
  'promptStrength': 7,
  'seed': 4523184,
  'status': 'finished',
  'steps': 50,
  'width': 512},
 {'createdAt': '2023-03-20T08:45:42.911424+00:00',
  'height